In [1]:
%%cmd
pip install "C:\Users\user\anaconda3\Dlib_Windows_Python3.x-main\dlib-19.24.99-cp312-cp312-win_amd64.whl"

Microsoft Windows [Version 10.0.19045.5247]
(c) Microsoft Corporation. All rights reserved.

C:\Users\user\anaconda3\Major Project\gaze detection>pip install "C:\Users\user\anaconda3\Dlib_Windows_Python3.x-main\dlib-19.24.99-cp312-cp312-win_amd64.whl"
Processing c:\users\user\anaconda3\dlib_windows_python3.x-main\dlib-19.24.99-cp312-cp312-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

C:\Users\user\anaconda3\Major Project\gaze detection>

In [2]:
!pip install opencv-python
!pip install pyglet

In [5]:
'''first we will integrate virtual keyboard and control that virtual keyboard using our eye movements with the help of Dlib which 
contains 68 landmark points for accurate left eye(36-41) and right eye(42-47) detection. We have also used Pyglet module to 
integrate sounds on opening left and right part of the keyboard and on pressing a key on virtual keyboard'''

'first we will integrate virtual keyboard and control that virtual keyboard using our eye movements with the help of Dlib which \ncontains 68 landmark points for accurate left eye(36-41) and right eye(42-47) detection. We have also used Pyglet module to \nintegrate sounds on opening left and right part of the keyboard and on pressing a key on virtual keyboard'

In [7]:
import cv2
import numpy as np
import dlib
from math import hypot
import pyglet
import time

sound = pyglet.media.load(r"C:\Users\user\anaconda3\Major Project\sound.wav", streaming=False)
left_sound = pyglet.media.load(r"C:\Users\user\anaconda3\Major Project\left.wav", streaming=False)
right_sound = pyglet.media.load(r"C:\Users\user\anaconda3\Major Project\right.wav", streaming=False)

cap = cv2.VideoCapture(0)
board = np.zeros((300, 1400), np.uint8)
board[:] = 255

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"C:\Users\user\anaconda3\Major Project\shape_predictor_68_face_landmarks.dat")

keyboard = np.zeros((600, 1000, 3), np.uint8)
keys_set_1 = {0: "Q", 1: "W", 2: "E", 3: "R", 4: "T",
              5: "A", 6: "S", 7: "D", 8: "F", 9: "G",
              10: "Z", 11: "X", 12: "C", 13: "V", 14: "<"}
keys_set_2 = {0: "Y", 1: "U", 2: "I", 3: "O", 4: "P",
              5: "H", 6: "J", 7: "K", 8: "L", 9: "_",
              10: "V", 11: "B", 12: "N", 13: "M", 14: "<"}

def draw_letters(letter_index, text, letter_light):
    if letter_index == 0:
        x = 0
        y = 0
    elif letter_index == 1:
        x = 200
        y = 0
    elif letter_index == 2:
        x = 400
        y = 0
    elif letter_index == 3:
        x = 600
        y = 0
    elif letter_index == 4:
        x = 800
        y = 0
    elif letter_index == 5:
        x = 0
        y = 200
    elif letter_index == 6:
        x = 200
        y = 200
    elif letter_index == 7:
        x = 400
        y = 200
    elif letter_index == 8:
        x = 600
        y = 200
    elif letter_index == 9:
        x = 800
        y = 200
    elif letter_index == 10:
        x = 0
        y = 400
    elif letter_index == 11:
        x = 200
        y = 400
    elif letter_index == 12:
        x = 400
        y = 400
    elif letter_index == 13:
        x = 600
        y = 400
    elif letter_index == 14:
        x = 800
        y = 400

    width = 200
    height = 200
    th = 3 

    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_th = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_th)[0]
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x
    text_y = int((height + height_text) / 2) + y

    if letter_light is True:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (255, 255, 255), -1)
        cv2.putText(keyboard, text, (text_x, text_y), font_letter, font_scale, (51, 51, 51), font_th)
    else:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (51, 51, 51), -1)
        cv2.putText(keyboard, text, (text_x, text_y), font_letter, font_scale, (255, 255, 255), font_th)

def draw_menu():
    rows, cols, _ = keyboard.shape
    th_lines = 4 # thickness lines
    cv2.line(keyboard, (int(cols/2) - int(th_lines/2), 0),(int(cols/2) - int(th_lines/2), rows),
             (51, 51, 51), th_lines)
    cv2.putText(keyboard, "LEFT", (80, 300), font, 6, (255, 255, 255), 5)
    cv2.putText(keyboard, "RIGHT", (80 + int(cols/2), 300), font, 6, (255, 255, 255), 5)

def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_lenght / ver_line_lenght
    return ratio

def eyes_contour_points(facial_landmarks):
    left_eye = []
    right_eye = []
    for n in range(36, 42):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        left_eye.append([x, y])
    for n in range(42, 48):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        right_eye.append([x, y])
    left_eye = np.array(left_eye, np.int32)
    right_eye = np.array(right_eye, np.int32)
    return left_eye, right_eye

def get_gaze_ratio(eye_points, facial_landmarks):
    left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)

    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)

    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])

    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape
    left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0: height, int(width / 2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white
    return gaze_ratio

frames = 0
letter_index = 0
blinking_frames = 0
frames_to_blink = 6
frames_active_letter = 9


text = ""
keyboard_selected = "left"
last_keyboard_selected = "left"
select_keyboard_menu = True
keyboard_selection_frames = 0

while True:
    _, frame = cap.read()
    rows, cols, _ = frame.shape
    keyboard[:] = (26, 26, 26)
    frames += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame[rows - 50: rows, 0: cols] = (255, 255, 255)

    if select_keyboard_menu is True:
        draw_menu()

    if keyboard_selected == "left":
        keys_set = keys_set_1
    else:
        keys_set = keys_set_2
    active_letter = keys_set[letter_index]

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        left_eye, right_eye = eyes_contour_points(landmarks)

        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2

        cv2.polylines(frame, [left_eye], True, (0, 0, 255), 2)
        cv2.polylines(frame, [right_eye], True, (0, 0, 255), 2)


        if select_keyboard_menu is True:
            gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2

            if gaze_ratio <= 0.9:
                keyboard_selected = "right"
                keyboard_selection_frames += 1
                if keyboard_selection_frames == 15:
                    select_keyboard_menu = False
                    right_sound.play()
                    frames = 0
                    keyboard_selection_frames = 0
                if keyboard_selected != last_keyboard_selected:
                    last_keyboard_selected = keyboard_selected
                    keyboard_selection_frames = 0
            else:
                keyboard_selected = "left"
                keyboard_selection_frames += 1
                if keyboard_selection_frames == 15:
                    select_keyboard_menu = False
                    left_sound.play()
                    frames = 0
                if keyboard_selected != last_keyboard_selected:
                    last_keyboard_selected = keyboard_selected
                    keyboard_selection_frames = 0

        else:
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1

                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<" and active_letter != "_":
                        text += active_letter
                    if active_letter == "_":
                        text += " "
                    sound.play()
                    select_keyboard_menu = True

            else:
                blinking_frames = 0


  
    if select_keyboard_menu is False:
        if frames == frames_active_letter:
            letter_index += 1
            frames = 0
        if letter_index == 15:
            letter_index = 0
        for i in range(15):
            if i == letter_index:
                light = True
            else:
                light = False
            draw_letters(i, keys_set[i], light)

    
    cv2.putText(board, text, (80, 100), font, 9, 0, 3)

   
    percentage_blinking = blinking_frames / frames_to_blink
    loading_x = int(cols * percentage_blinking)
    cv2.rectangle(frame, (0, rows - 50), (loading_x, rows), (51, 51, 51), -1)


    cv2.imshow("Frame", frame)
    cv2.imshow("Virtual keyboard", keyboard)
    cv2.imshow("Board", board)

    key = cv2.waitKey(1)
    # Exit the loop when the user presses the 'esc' key
    if key == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

In [9]:
'''Now we will map gaze coordinates to screen coordinates using PyAutoGUI to control cursor 
and Mediapipe facemesh for detecting iris relative landmark position.
When we move the head on webcam the position of cursor changes accordingly''' 

'Now we will map gaze coordinates to screen coordinates using PyAutoGUI to control cursor \nand Mediapipe facemesh for detecting iris relative landmark position.\nWhen we move the head on webcam the position of cursor changes accordingly'

In [11]:
!pip install mediapipe --user

In [21]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time

cam = cv2.VideoCapture(0)
face_mesh = mp.solutions.face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
screen_w, screen_h = pyautogui.size()

smooth_x, smooth_y = 0, 0
alpha = 0.5  
speed_scale = 1.5  

blink_threshold = 0.18 
blink_detected = False  
cursor_static = False  

def calculate_eye_aspect_ratio(eye_landmarks):
    vertical_1 = np.linalg.norm(
        np.array([eye_landmarks[1].x, eye_landmarks[1].y]) -
        np.array([eye_landmarks[5].x, eye_landmarks[5].y])
    )
    vertical_2 = np.linalg.norm(
        np.array([eye_landmarks[2].x, eye_landmarks[2].y]) -
        np.array([eye_landmarks[4].x, eye_landmarks[4].y])
    )
    horizontal = np.linalg.norm(
        np.array([eye_landmarks[0].x, eye_landmarks[0].y]) -
        np.array([eye_landmarks[3].x, eye_landmarks[3].y])
    )
    return (vertical_1 + vertical_2) / (2.0 * horizontal)

while True:
    _, frame = cam.read()
    frame = cv2.flip(frame, 1)  
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    if landmark_points:
        landmarks = landmark_points[0].landmark
        right_iris = landmarks[474:478]

    
        iris_x = sum([landmark.x for landmark in right_iris]) / 4
        iris_y = sum([landmark.y for landmark in right_iris]) / 4
        scaled_iris_x = min(max(iris_x * speed_scale, 0), 1)
        scaled_iris_y = min(max(iris_y * speed_scale, 0), 1)

       
        left_eye = [landmarks[i] for i in [362, 385, 387, 263, 373, 380]]
        right_eye = [landmarks[i] for i in [33, 160, 158, 133, 153, 144]]

        left_ear = calculate_eye_aspect_ratio(left_eye)
        right_ear = calculate_eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0

        print(f"Left EAR: {left_ear}, Right EAR: {right_ear}, Avg EAR: {avg_ear}")

        if avg_ear < blink_threshold:
            if not blink_detected:  
                blink_detected = True
                cursor_static = True  
                print("Blink detected! Clicking...")
                pyautogui.click() 
                time.sleep(0.5)  
        else:
            blink_detected = False  
            cursor_static = False  

        if not cursor_static:
            screen_x = int(screen_w * scaled_iris_x)
            screen_y = int(screen_h * scaled_iris_y)

            smooth_x = smooth_x + alpha * (screen_x - smooth_x)
            smooth_y = smooth_y + alpha * (screen_y - smooth_y)

            pyautogui.moveTo(int(smooth_x), int(smooth_y))

    cv2.putText(frame, f"EAR: {avg_ear:.3f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Eye Controlled Mouse', frame)
    # Exit the loop when the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit with 'q'
        break

cam.release()
cv2.destroyAllWindows()


Left EAR: 0.4213617925357786, Right EAR: 0.4409594378429142, Avg EAR: 0.4311606151893464
Left EAR: 0.43646064739485013, Right EAR: 0.3811869606324347, Avg EAR: 0.4088238040136424
Left EAR: 0.19026953829349627, Right EAR: 0.15088196753672053, Avg EAR: 0.1705757529151084
Blink detected! Clicking...
Left EAR: 0.4193454806748247, Right EAR: 0.3815928355542963, Avg EAR: 0.4004691581145605
Left EAR: 0.4292531543945045, Right EAR: 0.4097611914741357, Avg EAR: 0.4195071729343201
Left EAR: 0.3969556916858902, Right EAR: 0.35270453778026306, Avg EAR: 0.3748301147330766
Left EAR: 0.40570812772154813, Right EAR: 0.37982552516219725, Avg EAR: 0.39276682644187266
Left EAR: 0.3836311516229593, Right EAR: 0.3083908606185683, Avg EAR: 0.3460110061207638
Left EAR: 0.3827168483455432, Right EAR: 0.3510992686924543, Avg EAR: 0.36690805851899877
Left EAR: 0.40916411167914307, Right EAR: 0.33702276868279424, Avg EAR: 0.3730934401809687
Left EAR: 0.4155985672402129, Right EAR: 0.3202146216317438, Avg EAR: 0.

In [17]:
#This code provides scroll functionality ,It scrolls up and down when our eyes move to top 50% and below 50% area of the webcam.

In [19]:
import cv2
import dlib
import pyautogui
import numpy as np

cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"C:\Users\user\anaconda3\Major Project\shape_predictor_68_face_landmarks.dat")

screen_width, screen_height = pyautogui.size()

buffer_margin = 10

def get_eye_landmarks(landmarks, eye_points):
    eye_points = np.array([landmarks.part(i) for i in eye_points])
    return eye_points

def get_gaze_ratio(eye_points, landmarks):
    eye = get_eye_landmarks(landmarks, eye_points)
    left = eye[0].x
    right = eye[3].x
    top = eye[1].y
    bottom = eye[5].y
    return (left + right) / 2

def get_vertical_gaze(eye_points, landmarks):
    eye = get_eye_landmarks(landmarks, eye_points)
    top = eye[1].y
    bottom = eye[5].y
    return (top + bottom) / 2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)


        left_eye_vertical = get_vertical_gaze([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_vertical = get_vertical_gaze([42, 43, 44, 45, 46, 47], landmarks)

        vertical_gaze = (left_eye_vertical + right_eye_vertical) / 2

        vertical_percentage = (vertical_gaze / frame.shape[0]) * 100  

        if vertical_percentage < 50:  
            pyautogui.scroll(-5) 
        elif vertical_percentage > 50:  
            pyautogui.scroll(5) 
        elif vertical_percentage == 50:
            pyautogui.scroll(0)  

        cv2.putText(frame, f"Vertical Gaze: {vertical_percentage:.2f}%", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    cv2.imshow("Gaze Controlled Scrolling", frame)

    # Exit the loop when the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
